In [1]:
%matplotlib inline
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import sounddevice as sd
#__import__("python-sounddevice")

In [2]:
### Independent Components Analysis
###
### This program requires a working installation of:
###
### On Mac:
###     1. portaudio: On Mac: brew install portaudio
###     2. sounddevice: pip install sounddevice
###
### On windows:
###      pip install pyaudio sounddevice
###

In [3]:
Fs = 11025

def normalize(dat):
    return 0.99 * dat / np.max(np.abs(dat))

In [4]:
def load_data():
    mix = np.loadtxt('data/mix.dat')
    return mix

In [5]:
def play(vec):
    sd.play(vec, Fs, blocking=True)

In [6]:
def sigmoid(x):
    """
    Compute the sigmoid function for the input here.
    """
    ### YOUR CODE HERE
    pos_mask = (x >= 0)
    neg_mask = ~pos_mask
    s = np.empty_like(x)
    s[pos_mask] = 1 / (1 + np.exp(-x[pos_mask]))
    
    e_x_neg = np.exp(x[neg_mask])
    s[neg_mask] = e_x_neg / (1 + e_x_neg)

    ### END YOUR CODE
    return s

In [7]:
def unmixer(X):
    M, N = X.shape
    W = np.eye(N)
    anneal = [0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.02, 0.02, 0.01, 0.01,
              0.005, 0.005, 0.002, 0.002, 0.001, 0.001]
    print('Separating tracks ...')
    ######## Your code here ##########
    p = np.random.permutation(X.shape[0])
    X_shuffled = X[p, :]
    #X_shuffled = X
    len_anneal = len(anneal)
    for anneal_rate in anneal:
        for i, x in enumerate(X_shuffled):
            x = x.reshape((x.shape[0], 1))
            d_W = np.dot( ( 1 - 2 * sigmoid(np.dot(W, x)) ), x.T ) + np.linalg.inv(W.T)
            W += anneal_rate * d_W
    ###################################
    return W

In [8]:
def unmix(X, W):
    S = np.zeros(X.shape)

    ######### Your code here ##########
    S = np.dot(W, X.T)
    ##################################
    return S

In [9]:
def main():
    X = normalize(load_data())
    for i in range(X.shape[1]):
        print('Playing mixed track %d' % i)
        play(X[:, i])
    
    W = unmixer(X)
    S = normalize(unmix(X, W))

    S_T = S.T
    for i in range(S_T.shape[1]):
        print('Playing separated track %d' % i)
        play(S_T[:, i])

In [10]:
if __name__ == '__main__':
    main()


Playing mixed track 0
Playing mixed track 1
Playing mixed track 2
Playing mixed track 3
Playing mixed track 4
Separating tracks ...
Playing separated track 0
Playing separated track 1
Playing separated track 2
Playing separated track 3
Playing separated track 4
